##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFF で ClientData を使用する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/working_with_client_data">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/working_with_client_data.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/working_with_client_data.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TFF でモデル化された連合計算には、クライアント（ユーザーなど）によってキー設定されたデータセットの概念が不可欠です。 TFF は、この概念を抽象化するためのインターフェース [`tff.simulation.datasets.ClientData`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData) を提供します。TFF がホストするデータセット([stackoverflow](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow), [shakespeare](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/shakespeare)、[emnist](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist)、[cifar100](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/cifar100)、 [gldv2](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/gldv2)) は、すべてこのインターフェースを実装します。

独自のデータセットを使用して連合学習に取り組んでいる場合、TFF は、`ClientData` インターフェースを実装するか、TFF のヘルパー関数の 1 つを使用して `ClientData` を生成することを強くお勧めします。これは、[`tff.simulation.datasets.ClientData.from_clients_and_fn`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData#from_clients_and_fn) などのディスク上のデータを示します。

TFF のエンドツーエンドの例のほとんどは `ClientData` オブジェクトで始まるため、カスタムデータセットで `ClientData` インターフェースを実装すると、TFF で記述された既存のコードを簡単に調べることができます。さらに、`ClientData` が構築する `tf.data.Datasets` を直接繰り返して、`numpy` 配列の構造を生成できます。

シミュレーションを多くのマシンにスケールアップしたり、それらをデプロイしたりする場合は、いくつかのパターンを使用して作業を楽にすることができます。以下では、`ClientData` と TFF を使用して、小規模な反復から大規模な実験から本番環境へのデプロイエクスペリエンスを可能な限りスムーズにする方法をいくつか紹介します。

## クライアントデータを TTF に渡すために使用するパターン

TFF の `ClientData` の 2 つの使用法について詳しく説明します。以下の 2 つのカテゴリのいずれかに当てはまる場合は、一方を優先することでしょう。より微妙な選択を行うためには、それぞれの長所と短所をより詳細に理解する必要があります。

- ローカルマシンでできるだけ早く反復したい。TFF の分散ランタイムを簡単に利用できる必要はない。

    - `tf.data.Datasets` を TFF に直接渡します。
    - これにより、`tf.data.Dataset`オブジェクトを使用して必須のプログラムを作成し、それらを任意に処理できます。
    - 以下のオプションよりも柔軟性があります。ロジックをクライアントにプッシュするには、このロジックがシリアル化可能である必要があります。

- 連合計算を TFF のリモートランタイムで実行したい、または近い将来に実行する予定がある場合。

    - この場合、データセットの構築と前処理をクライアントにマッピングする必要があります。
    - これにより、`client_ids` のリストを連合計算に直接渡すことになります。

- データセットの構築と前処理をクライアントにプッシュすることにより、シリアル化のボトルネックを回避し、数百から数千のクライアントでパフォーマンスを大幅に向上できます。

In [ ]:
#@title Set up open-source environment
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow_federated

In [1]:
#@title Import packages
import collections
import time

import tensorflow as tf
import tensorflow_federated as tff

## ClientData オブジェクトの操作

まず、TFF の EMNIST `ClientData` を読み込んで調べてみましょう。


In [2]:
client_data, _ = tff.simulation.datasets.emnist.load_data()

最初のデータセットを調べると、`ClientData` にどのようなサンプルがあるかがわかります。

In [3]:
first_client_id = client_data.client_ids[0]
first_client_dataset = client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)
# This information is also available as a `ClientData` property:
assert client_data.element_type_structure == first_client_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


データセットは、`pixels` と `label` キーを持つ `collections.OrderedDict` オブジェクトを生成することに注意してください。pixels は形状 `[28, 28]` のテンソルです。入力をフラット化して `[784]` の形状にするための 1 つの方法として、前処理関数を `ClientData` オブジェクトに適用することができます。

In [4]:
def preprocess_dataset(dataset):
  """Create batches of 5 examples, and limit to 3 batches."""

  def map_fn(input):
    return collections.OrderedDict(
        x=tf.reshape(input['pixels'], shape=(-1, 784)),
        y=tf.cast(tf.reshape(input['label'], shape=(-1, 1)), tf.int64),
    )

  return dataset.batch(5).map(
      map_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE).take(5)


preprocessed_client_data = client_data.preprocess(preprocess_dataset)

# Notice that we have both reshaped and renamed the elements of the ordered dict.
first_client_dataset = preprocessed_client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


さらに、シャッフルなど、より複雑な（場合によってはステートフルな）前処理を実行したい場合があります。

In [5]:
def preprocess_and_shuffle(dataset):
  """Applies `preprocess_dataset` above and shuffles the result."""
  preprocessed = preprocess_dataset(dataset)
  return preprocessed.shuffle(buffer_size=5)

preprocessed_and_shuffled = client_data.preprocess(preprocess_and_shuffle)

# The type signature will remain the same, but the batches will be shuffled.
first_client_dataset = preprocessed_and_shuffled.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


## `tff.Computation` とのインターフェース

`ClientData` オブジェクトを使用していくつかの基本的な操作を実行できるようになったので、データを `tff.Computation` にフィードする準備が整いました。[連合平均化](https://arxiv.org/abs/1602.05629)を実装する [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess) を定義し、データを渡すさまざまな方法を検討します。

In [6]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
  ])
  return tff.learning.models.from_keras_model(
      model,
      # Note: input spec is the _batched_ shape, and includes the 
      # label tensor which will be passed to the loss function. This model is
      # therefore configured to accept data _after_ it has been preprocessed.
      input_spec=collections.OrderedDict(
          x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
          y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01))

この `IterativeProcess` での作業を開始する前に、`ClientData` のセマンティクスに関する 1 つのコメントが必要です。`ClientData` オブジェクトは、連合トレーニングに使用できる*全*母集団を表します。これは、通常、[実稼働の連合システムの実行環境では使用できず](https://arxiv.org/abs/1902.01046)、シミュレーションのみに使用できます。`ClientData` は、連合計算を完全にバイパスし、[`ClientData.create_tf_dataset_from_all_clients`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData?hl=en#create_tf_dataset_from_all_clients) を介して通常どおりサーバー側モデルをトレーニングする機能をユーザーに提供します。

TFF のシミュレーション環境により、研究者は外側のループを完全に制御できます。特に、これは、クライアントの可用性、クライアントのドロップアウトなどの考慮事項がユーザーまたは Python ドライバースクリプトにより対処される必要があることを意味します。たとえば、`ClientData's` `client_ids` のサンプリング分布を調整することで、より多くのデータ（およびそれに対応してより長時間実行されるローカル計算）を持つユーザーがより低い確率で選択されるようにクライアントのドロップアウトをモデル化できます。

ただし、実際の連合システムでは、モデルトレーナーがクライアントを明示的に選択することはできません。クライアントの選択は、連合計算を実行しているシステムに委任されます。

### `tf.data.Datasets` を直接 TFF に渡す

`ClientData` と `IterativeProcess` の間のインターフェースに使用できるオプションの 1 つとして、Python で `tf.data.Datasets` を作成し、これらのデータセットを TFF に渡すことができます。

前処理された `ClientData` を使用する場合、生成されるデータセットは、上記で定義されたモデルで期待される適切な型であることに注意してください。

In [7]:
selected_client_ids = preprocessed_and_shuffled.client_ids[:10]

preprocessed_data_for_clients = [
    preprocessed_and_shuffled.create_tf_dataset_for_client(
        selected_client_ids[i]) for i in range(10)
]

state = trainer.initialize()
for _ in range(5):
  t1 = time.time()
  result = trainer.next(state, preprocessed_data_for_clients)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.934802532196045, round time 2.5420753955841064
loss 3.350963830947876, round time 0.45527172088623047
loss 3.1382687091827393, round time 0.47087883949279785
loss 3.0774152278900146, round time 0.4089682102203369
loss 2.9193594455718994, round time 0.3964221477508545


ただし、この方法を使用すると、***マルチマシンシミュレーション***に簡単に移行できなくなります。ローカルの TensorFlow ランタイムで構築するデータセットは、*周囲の Python 環境から状態をキャプチャする*ことができ、使用できなくなった状態を参照しようとすると、シリアル化または逆シリアル化に失敗します。これは、TensorFlow の `tensor_util.cc` からの不可解なエラーなどで明らかになる可能性があります。

```
Check failed: DT_VARIANT == input.dtype() (21 vs. 20)
```

### クライアント上でのマッピング構築と前処理

この問題を回避するために、TFF は、データセットのインスタンス化と前処理を*各クライアントでローカルに発生するものと見なすこと*をユーザーに推奨しています。また、TFF のヘルパーまたは `federated_map` を使用して、各クライアントでこの前処理コードを明示的に実行することを推薦しています。

概念的には、この理由は明らかです。TFF のローカルランタイムでは、連合オーケストレーション全体が単一のマシンで行われているため、クライアントは「誤った」場合にのみグローバル Python 環境にアクセスします。同様の考え方が TFF のクロスプラットフォームで、常にシリアル化可能な機能哲学を生み出していることは注目に値します。

TFF は、`ClientData` の属性 `dataset_computation` を介して、このような変更を簡単にします。`tff.Computation` は、`client_id` を受け取り、関連する `tf.data.Dataset` を返します。

`preprocess` は単に`dataset_computation` で機能することに注意してください。前処理された `ClientData` の `dataset_computation` 属性には、先ほど定義した前処理パイプライン全体が組み込まれています。

In [8]:
print('dataset computation without preprocessing:')
print(client_data.dataset_computation.type_signature)
print('\n')
print('dataset computation with preprocessing:')
print(preprocessed_and_shuffled.dataset_computation.type_signature)

dataset computation without preprocessing:
(string -> <label=int32,pixels=float32[28,28]>*)


dataset computation with preprocessing:
(string -> <x=float32[?,784],y=int64[?,1]>*)


`dataset_computation` を呼び出して、Python ランタイムで eager データセットを受け取ることができます。しかし、このアプローチの真の力は、反復プロセスまたは別の計算で構成するときに発揮され、グローバルな eager ランタイムでこれらのデータセットを実現することを回避できることです。TFF は、これを正確に行うために使用できるヘルパー関数 [`tff.simulation.compose_dataset_computation_with_iterative_process`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/compose_dataset_computation_with_iterative_process) を提供します。

In [9]:
trainer_accepting_ids = tff.simulation.compose_dataset_computation_with_iterative_process(
    preprocessed_and_shuffled.dataset_computation, trainer)

この `tff.templates.IterativeProcesses` と上記のものはどちらも同じように実行されます。ただし、前者は前処理されたクライアントデータセットを受け入れ、後者はクライアント ID を表す文字列を受け入れ、データセットの構築と前処理の両方を本体で処理します。実際、`state` は 2 つの間で渡すことができます。

In [10]:
for _ in range(5):
  t1 = time.time()
  result = trainer_accepting_ids.next(state, selected_client_ids)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.6114611625671387, round time 1.4935951232910156
loss 2.612247943878174, round time 0.30751872062683105
loss 2.8368589878082275, round time 0.3043978214263916
loss 2.6863903999328613, round time 0.3107311725616455
loss 2.6816341876983643, round time 0.4325370788574219


### 多数のクライアントへのスケーリング

`trainer_accepting_ids` は、TFF のマルチマシンランタイムですぐに使用でき、`tf.data.Datasets` とコントローラの実体化を回避します（したがって、それらをシリアル化してワーカーに送信します）。

これにより、特に多数のクライアントでの分散シミュレーションが大幅にスピードアップし、中間集約が可能になり、同様のシリアル化/逆シリアル化のオーバーヘッドを回避できます。


### オプションの詳細: TFF で前処理ロジックを手動で作成する

TFF は、ゼロから構成できるように設計されています。TFF のヘルパーによって先ほど実行されたような構成は、完全に制御できます。トレーナー自身の `next` を使用して、定義した前処理計算を手動で作成することもできます。

In [11]:
selected_clients_type = tff.FederatedType(preprocessed_and_shuffled.dataset_computation.type_signature.parameter, tff.CLIENTS)

@tff.federated_computation(trainer.next.type_signature.parameter[0], selected_clients_type)
def new_next(server_state, selected_clients):
  preprocessed_data = tff.federated_map(preprocessed_and_shuffled.dataset_computation, selected_clients)
  return trainer.next(server_state, preprocessed_data)

manual_trainer_with_preprocessing = tff.templates.IterativeProcess(initialize_fn=trainer.initialize, next_fn=new_next)

実際、これは、私たちが使用したヘルパーが内部で行っていることです（さらに、適切な型のチェックと操作を実行しています）。同じロジックを少し異なって表現することもできます。<br>`preprocess_and_shuffle` を `tff.Computation` にシリアル化し、 `federated_map` を 1 つのステップに分解します。<br>これは前処理されていないデータセットを構築し、別のデータセットは各クライアントで `preprocess_and_shuffle` を実行します。

このより手動のパスにより、TFF のヘルパー（モジュロパラメータ名）と同じ型アノテーションを使用して計算が行われることを確認できます。

In [12]:
print(trainer_accepting_ids.next.type_signature)
print(manual_trainer_with_preprocessing.next.type_signature)

(<state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,client_data={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,metrics=<distributor=<>,client_work=<train=<sparse_categorical_accuracy=float32,loss=float32,num_examples=int64,num_batches=int64>>,aggregator=<mean_value=<>,mean_weight=<>>,finalizer=<>>@SERVER>)
(<server_state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,selected_clients={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggre